In [ ]:
# HIDDEN
import matplotlib
#matplotlib.use('Agg')
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import math
import scipy.stats as stats
plt.style.use('fivethirtyeight')

#Created by Kaiqian Zhu(Classifier, Compare two samples) && Mariah Rogers(Two Categorical Distribution, A/B Testing)

# Classifier

In [ ]:
iris = Table.read_table('iris.csv')
iris.show(10)


### Question 1: List all the features and classes on the table above. (you might use code to help you find all classes)

**SOLUTION:**
* **features**: sepal_length, sepal_width, petal_length, petal_width
* **classes**: setosa, versicolor, virginica

In [ ]:
# Now we assign a color to each class

color_table = Table().with_columns(
    'species', make_array('setosa','versicolor','virginica'),
    'Color', make_array('darkblue', 'gold', 'lightblue')
)

iris = iris.join('species', color_table).drop(6,7)

iris


### Question 2: Create a scatter graph using only the first 2 features. Is it enough to distinguish the three species?

**SOLUTION:** No, versicolor, virginica have similar measure on these two features

In [ ]:
iris.scatter('sepal_length','sepal_width',colors ='species')

### Question 3: Now we have a unknown iris with sepal_length = 5.1 and  sepal_width= 4.2. Can you classify it? Also, what about an unknown iris with sepal_length = 6 and  sepal_width= 3?

**SOLUTION:**
* Yes, it is most likely a setosa.
* No, we need more features.

In [ ]:
#Here we make a 3d scatter graph adding one more feature - petal_length.

ax = plt.figure(figsize=(8,8)).add_subplot(111, projection='3d')
ax.scatter(iris.column('sepal_length'), 
           iris.column('sepal_width'), 
           iris.column('petal_length'), 
           c=iris.column('Color'));

# Can you see the light blue dots and gold dots are separted? What if we add more features?

### Question 4: Now, can you classify an unknown iris with sepal_length = 6, sepal_width= 3, and petal_length = 5.7?

**SOLUTION:** Yes, it is most likely a virginica.

### Question 5: Calculate the distance between iris.row(0) and iris.row(100), iris.row(0) and iris.row(1) using the distance formula.

In [ ]:
#Solution: 
def distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

iris_dist = iris.drop(0,5)
pt1 = np.array(iris_dist.row(0))
pt2 = np.array(iris_dist.row(100))
pt3 = np.array(iris_dist.row(1))
print(distance(pt1,pt2))
distance(pt1,pt3)


# Comparing Two Samples

In [ ]:
iris_petal_width = iris.select('species', 'petal_width')
iris_petal_width

In [ ]:
iris_petal_width_pivot = iris_petal_width.pivot('species', 'petal_width')
iris_petal_width_pivot

In [ ]:
def proportions(array):
    return array/np.sum(array)

iris_dists = iris_petal_width_pivot.select(0).with_columns(
   'setosa', proportions(iris_petal_width_pivot.column('setosa')),
    'versicolor', proportions(iris_petal_width_pivot.column('versicolor')),
    'virginica', proportions(iris_petal_width_pivot.column('virginica'))
)

iris_dists.barh('petal_width')

### Question 1: Do the 3 distributions look the same or very different? 

**SOLUTION:** They look very different and we should be able to contruct a very good classifier based on that.

### Question 2: Contruct a simple but accurate classifier based on the observation you have for these distinct distributions.
#### Describe how you would classify a data point given simple restraints on feature values.

**SOLUTION:** "If the petal_width > 0 and < 1, say the Class is setosa;If the petal_width >= 1 and < 1.7, say the Class is versicolor; If the petal_width >= 1,7 and < 2.6, say the Class is virginica"

## 4. A Potpourri of Tests


#### Question 1
Many scientific disciplines use 5% as a standard cutoff for rejecting the null hypothesis when conducting hypothesis tests.  Suppose for sake of argument that every scientific paper hinges on exactly one hypothesis test with a 5% cutoff.  After learning about hypothesis testing, Thomas despairs about the state of scientific research, wondering:

> "Doesn't this mean that 5% of all scientific papers are wrong?"

Give one reason why Thomas's worry isn't quite accurate.

**SOLUTION:** A 5% cutoff means that 5% of tests *conducted on true null hypotheses* would falsely reject those hypothesis.  So if, say, 25% of null hypotheses tested were false, then scientists would only only falsely reject null hypotheses $75\% \times 5\% = 3.75\%$ of the time.  Usually the null hypothesis in a scientific paper is that the researcher's idea is incorrect, and probably not *all* ideas tested in scientific papers are incorrect!  (Note that this ignores the chance of falsely accepting a false null hypothesis.  But the chance of that happening is not necessarily 5%, so the overall chance of error is not necessarily 5%.)

#### Question 2
Many scientists hope to make exciting and unintuitive discoveries.  Often the null hypothesis in an hypothesis test is something boring ("the sky is blue"), while the alternative is surprising ("the sky is not blue!").

Suppose a scientist has an exciting but incorrect idea, so that their null hypothesis is *truly correct*.  When an hypothesis test is run on a sample of data, it fails to reject the null when using a 5% cutoff.  Disappointed but determined, the scientist gathers 10 more samples and runs the same test on each sample (running 10 separate hypothesis tests, each with a 5% cutoff).  If any of the tests rejects the null, the scientist publishes that one.  What is the chance that any of those tests rejects the null hypothesis?

**SOLUTION:** The chance to reject the null at least once is $1 - (1 - .05)^{10}$, or around 40%!

**Note:** The scientist in this scenario is acting very unethically, but (probably milder) forms of this [*publication bias*](https://en.wikipedia.org/wiki/Publication_bias) seem to be a real problem in science today.

#### Question 3
Explain one advantage and one disadvantage of using a very high confidence level like 99.9999% when computing a confidence interval.  Describe a situation where you would advise using such a high confidence level.

**SOLUTION:** Advantage: Most of the time, your interval will cover the population parameter.  Disadvantage: Your interval will be very wide, so you may not be able to make practically useful decisions from it.  I would advise using such a high confidence level in estimating the strength of a material that will go in an astronaut's space suit, because the cost of being wrong is very high.

#### Question 4
When completing a permutation test, what is TVD used for?

**SOLUTION:** TVD is used as our test statistic in a permutation test when we would like to compare to see whether two categorical distributions are the same.

## 5. Some general review
**Please explain your answers fully.**

### Conceptual
#### Question 1

For what types of situations/problems would we want to use a hypothesis test? For what situations/problems would we use a confidence interval? And for which would we do both?

**SOLUTION:** We would use a hypothesis test for times when we want to know if the world works one way, or another way. Examples would include: asking if a coin is fair or not, or asking if a customer is cheating or not.

We would use a confidence interval for times when we want to estimate a population parameter (side note: a population parameter is a number/descriptor about a population). Examples include estimate the population median annual household income, or estimating the true average height of US adults.

We would use both in a very specific situation. If you want to know whether a a descriptor is a certain number or not, you can set up your hypothesis test like:

> Null: The slope of the true regression line between X and Y is 0. 

> Alternative: the slope of the true regression line between X and Y is not 0. After you clearly define your hypothesis, you can test it by constructing a confidence interval that number (in this case the slope). If you confidence interval doesn’t contain the number (in this case 0), you reject the null hypothesis.



#### Question 2

Why do we need to construct a confidence interval for a population paramater in the first place? Why isn’t our sample statistic a good enough estimate?

**SOLUTION:** Our sample statistic isn’t a good enough estimate because since our sample was drawn at random, our sample could have come out differently. Then our sample statistic could have been different. So by itself, it’s not a good estimate. We can account for how the sample could have been different by bootstrapping the sample! By bootstrapping, and finding more sample statistics, we can account for the different possible values of the sample statistic, and provide a more accurate estimate for the population parameter.

### Practice
#### Question 3

I have a standard 52 card deck. I draw two cards without replacement. What is the chance that I draw an ace and a jack of hearts?

**SOLUTION:** $(4/52 * 1/51) + (1/52 * 4/51)$

There are two ways this event can happen: you either draw the ace first, and then the jack, or vice versa, so you have to add the probabilities of those events together. Also, since you are drawing without replacement, the number of cards to choose from decreases after the first draw.

**Question 4:**

I roll a fair six sided die. What’s the chance that I get the “1” face or the “2” face?

**SOLUTION:** (1/6) + (1/6) because the event can happened in one of two ways, so we need to account for both ways it can happen by adding the two probabilities.

#### Question 5

Write the code that would evaluate to a 95% confidence interval for the average height of US adults. Assume you have a table $sample\_heights$  that contained the heights of a randomly drawn 1000-person sample, with one column “height” that describes each person’s height in inches.

**Note:** You will probably not be able to run your code, since $sample\_heights$ is not defined.